# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

## TODO Explore data

In [2]:
# load in the data
azdias = pd.read_csv('../data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../data/Udacity_CUSTOMERS_052018.csv', sep=';')

print(azdias.shape)
azdias.head()

/home/gabor/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(891221, 366)


,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [3]:
# Read up data dictionary
df_dictionary = pd.read_excel('../data/DIAS Attributes - Values 2017.xlsx', skiprows=1, usecols=[1,2,3,4])
df_dictionary['Attribute'].fillna(method='ffill', inplace=True)
df_dictionary.loc[df_dictionary['Value']=='…', 'Type'] = 'Numerical'
df_dictionary.loc[~(df_dictionary['Value']=='…'), 'Type'] = 'Non-Numerical'

# list of possible values
possible_values = df_dictionary.groupby(by='Attribute')['Value'].apply(list)
data_type = df_dictionary[['Attribute', 'Type']].drop_duplicates()
print(len(possible_values), len(data_type))
data_type = data_type.merge(possible_values, left_on='Attribute', right_index=True).reset_index(drop=True)
print(len(data_type))
# missing values
df_missing = df_dictionary[df_dictionary['Meaning'].astype('str').str.contains('unknown')]
print(len(data_type))
data_type = data_type.merge(df_missing[['Attribute', 'Value']], left_on='Attribute', right_on='Attribute', how='left').reset_index(drop=True)
print(len(data_type))
data_type.rename(columns={'Value_x':'Values','Value_y':'Missing values'}, inplace=True)
data_type.head(2)

314 314
314
314
314


,Attribute,Type,Values,Missing values
0,AGER_TYP,Non-Numerical,"[-1, 0, 1, 2, 3]",-1
1,ALTERSKATEGORIE_GROB,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 9]","-1, 0"


In [4]:
cols_customers = customers.columns
cols_azdias = azdias.columns
cols_datadict = sorted(data_type['Attribute'])
print(len(cols_customers))
print(len(cols_azdias))
print(len(cols_datadict))

df_cols = pd.Series(sorted(cols_customers), name='customers').to_frame()
df_cols = df_cols.merge(pd.Series(sorted(cols_azdias), name='azdias').to_frame(), left_on='customers', right_on='azdias', how='outer')

# Deal with mispelled column names
#rename the _RZ
cols_datadict = [col[0:-3] if col[-3:]=='_RZ' else col for col in cols_datadict ]
df_cols.loc[df_cols['customers']=='D19_BUCH_CD'] = 'D19_BUCH'

# KBA13_CCM_1400_2500 -> KBA13_CCM_1401_2500
df_dict_cols = pd.Series(sorted(cols_datadict), name='dictionary').to_frame()
df_dict_cols.loc[df_dict_cols['dictionary']=='KBA13_CCM_1400_2500']='KBA13_CCM_1401_2500'

# CAMEO_DEUINTL_2015 -> CAMEO_INTL_2015
df_dict_cols.loc[df_dict_cols['dictionary']=='CAMEO_DEUINTL_2015']='CAMEO_INTL_2015'

# SOHO_FLAG -> SOHO_KZ
df_dict_cols.loc[df_dict_cols['dictionary']=='SOHO_FLAG']='SOHO_KZ'

df_cols = df_cols.merge(df_dict_cols, left_on='customers', right_on='dictionary', how='outer')

df_cols['All Cols']=df_cols.fillna(method='pad', axis=1)['dictionary']
len(df_cols)

369
366
314


374

In [58]:
data_type['Attribute'] = data_type['Attribute'].str.replace('_RZ', '', regex=True)
col_name_changes = {'KBA13_CCM_1400_2500':'KBA13_CCM_1401_2500',
                    'CAMEO_DEUINTL_2015':'CAMEO_INTL_2015',
                    'SOHO_FLAG': 'SOHO_KZ'}

data_type['Attribute'] = data_type['Attribute'].replace(col_name_changes)
# data_type.loc[data_type['Attribute']=='D19_BUCH_RZ', 'Attribute'] = 'KBA13_CCM_1401_2500'


In [85]:
azdias.columns = azdias.columns.str.replace('D19_BUCH_CD', 'D19_BUCH')


In [75]:
cols_to_drop = df_cols[df_cols.isnull().any(axis=1)]['All Cols'].values
print(len(cols_to_drop))
cols_to_drop[:20]

65


array(['AKT_DAT_KL', 'ALTERSKATEGORIE_FEIN', 'ALTER_KIND1', 'ALTER_KIND2',
       'ALTER_KIND3', 'ALTER_KIND4', 'ANZ_KINDER',
       'ANZ_STATISTISCHE_HAUSHALTE', 'ARBEIT', 'CJT_KATALOGNUTZER',
       'CJT_TYP_1', 'CJT_TYP_2', 'CJT_TYP_3', 'CJT_TYP_4', 'CJT_TYP_5',
       'CJT_TYP_6', 'CUSTOMER_GROUP', 'D19_KONSUMTYP_MAX',
       'D19_LETZTER_KAUF_BRANCHE', 'D19_SOZIALES'], dtype=object)

In [76]:
df_cols[df_cols['All Cols'].str.contains('CAMEO')]

,customers,azdias,dictionary,All Cols
18,CAMEO_DEUG_2015,CAMEO_DEUG_2015,CAMEO_DEUG_2015,CAMEO_DEUG_2015
19,CAMEO_DEU_2015,CAMEO_DEU_2015,CAMEO_DEU_2015,CAMEO_DEU_2015
20,CAMEO_INTL_2015,CAMEO_INTL_2015,CAMEO_INTL_2015,CAMEO_INTL_2015


## I'll need columns present in both dataset at the same time.

In [86]:
# Drop all non matching columns
cols_to_drop_azdias = [col for col in cols_to_drop if col in azdias.columns ]
print(len(cols_to_drop_azdias))
azdias_coldropped = azdias.drop(columns=cols_to_drop_azdias)

cols_to_drop_customers = [col for col in cols_to_drop if col in customers.columns ]
print(len(cols_to_drop_customers))
customers_coldropped = customers.drop(columns=cols_to_drop_customers)

57
60


## TODO Clean data

In [97]:
feat_info = pd.read_csv('../features.csv', index_col=0)

# Convert missing values to Nans
missing_values = pd.Series(feat_info['missing_or_unknown'].values, index=feat_info.index).to_dict()
# clean_df[clean_df.isin(missing_values)] = np.nan

In [99]:
missing_values
# feat_info

{'AGER_TYP': '[-1,0]',
 'ALTER_HH': '[0]',
 'ALTERSKATEGORIE_GROB': '[-1,0,9]',
 'ANREDE_KZ': '[-1,0]',
 'ANZ_HAUSHALTE_AKTIV': '[0]',
 'ANZ_HH_TITEL': '[]',
 'ANZ_KINDER': '[]',
 'ANZ_PERSONEN': '[]',
 'ANZ_STATISTISCHE_HAUSHALTE': '[]',
 'ANZ_TITEL': '[]',
 'ARBEIT': '[-1,9]',
 'BALLRAUM': '[-1]',
 'BIP_FLAG': '[-1]',
 'CAMEO_DEU_2015': '[XX]',
 'CAMEO_DEUG_2015': '[-1,X]',
 'CAMEO_INTL_2015': '[-1,XX]',
 'CJT_GESAMTTYP': '[0]',
 'D19_BANKEN_ANZ_12': '[]',
 'D19_BANKEN_ANZ_24': '[]',
 'D19_BANKEN_DATUM': '[]',
 'D19_BANKEN_DIREKT': '[]',
 'D19_BANKEN_GROSS': '[]',
 'D19_BANKEN_LOKAL': '[]',
 'D19_BANKEN_OFFLINE_DATUM': '[]',
 'D19_BANKEN_ONLINE_DATUM': '[]',
 'D19_BANKEN_ONLINE_QUOTE_12': '[]',
 'D19_BANKEN_REST': '[]',
 'D19_BEKLEIDUNG_GEH': '[]',
 'D19_BEKLEIDUNG_REST': '[]',
 'D19_BILDUNG': '[]',
 'D19_BIO_OEKO': '[]',
 'D19_BUCH_CD': '[]',
 'D19_DIGIT_SERV': '[]',
 'D19_DROGERIEARTIKEL': '[]',
 'D19_ENERGIE': '[]',
 'D19_FREIZEIT': '[]',
 'D19_GARTEN': '[]',
 'D19_GESAMT_ANZ_12':

In [110]:
data_type.loc[data_type['Type']=='Non-Numerical'].set_index('Attribute')['Missing values'].apply(list)

TypeError: 'int' object is not iterable

In [103]:
missing_data = data_type.loc[data_type['Type']=='Non-Numerical'].set_index('Attribute')['Missing values'].to_dict()
missing_data

{'AGER_TYP': -1,
 'ALTERSKATEGORIE_GROB': '-1, 0',
 'ALTER_HH': 0,
 'ANREDE_KZ': '-1, 0',
 'BALLRAUM': -1,
 'BIP_FLAG': -1,
 'CAMEO_DEUG_2015': -1,
 'CAMEO_DEU_2015': nan,
 'CAMEO_INTL_2015': -1,
 'CJT_GESAMTTYP': 0,
 'D19_BANKEN_ANZ_12': nan,
 'D19_BANKEN_ANZ_24': nan,
 'D19_BANKEN_DATUM': nan,
 'D19_BANKEN_DIREKT': nan,
 'D19_BANKEN_GROSS': nan,
 'D19_BANKEN_LOKAL': nan,
 'D19_BANKEN_OFFLINE_DATUM': nan,
 'D19_BANKEN_ONLINE_DATUM': nan,
 'D19_BANKEN_ONLINE_QUOTE_12': nan,
 'D19_BANKEN_REST': nan,
 'D19_BEKLEIDUNG_GEH': nan,
 'D19_BEKLEIDUNG_REST': nan,
 'D19_BILDUNG': nan,
 'D19_BIO_OEKO': nan,
 'D19_BUCH': nan,
 'D19_DIGIT_SERV': nan,
 'D19_DROGERIEARTIKEL': nan,
 'D19_ENERGIE': nan,
 'D19_FREIZEIT': nan,
 'D19_GARTEN': nan,
 'D19_GESAMT_ANZ_12': nan,
 'D19_GESAMT_ANZ_24': nan,
 'D19_GESAMT_DATUM': nan,
 'D19_GESAMT_OFFLINE_DATUM': nan,
 'D19_GESAMT_ONLINE_DATUM': nan,
 'D19_GESAMT_ONLINE_QUOTE_12': nan,
 'D19_HANDWERK': nan,
 'D19_HAUS_DEKO': nan,
 'D19_KINDERARTIKEL': nan,
 'D19_K

In [105]:
type(missing_data)

dict

In [106]:
azdias.isin(missing_data)

TypeError: only list-like or dict-like objects are allowed to be passed to DataFrame.isin(), you passed a 'int'

In [10]:
 #==

,Attribute,Type,Values,Missing values
0,AGER_TYP,Non-Numerical,"[-1, 0, 1, 2, 3]",-1
1,ALTERSKATEGORIE_GROB,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 9]","-1, 0"
2,ALTER_HH,Non-Numerical,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
3,ANREDE_KZ,Non-Numerical,"[-1, 0, 1, 2]","-1, 0"
4,ANZ_HAUSHALTE_AKTIV,Numerical,[…],NaN
...,...,...,...,...
309,WOHNDAUER_2008,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","-1, 0"
310,WOHNLAGE,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 5, 7, 8]",-1
311,WACHSTUMSGEBIET_NB,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 5]","-1, 0"
312,W_KEIT_KIND_HH,Non-Numerical,"[-1, 0, 1, 2, 3, 4, 5, 6]","-1, 0"


rename the azdias columns

In [89]:
# Loop through columns, check if values are in the right range and convert missing values to NaN
for col in azdias_coldropped.columns:
    
    if not data_type.loc[data_type['Attribute']==col, 'Type'].iloc[0]=='Numerical':
    
        print('{}: {}'.format(col, azdias_coldropped[col].unique()))


AGER_TYP: [-1  2  3  0  1]
ALTER_HH: [nan  0. 17. 13. 20. 10. 14. 16. 21. 11. 19. 15.  9. 18.  8.  7. 12.  4.
  3.  6.  5.  2.  1.]
BALLRAUM: [nan  6.  2.  4.  3.  7.  1.  5.]
CAMEO_DEU_2015: [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C' 'XX']
CAMEO_DEUG_2015: [nan 8.0 4.0 2.0 6.0 1.0 9.0 5.0 7.0 3.0 '4' '3' '7' '2' '8' '9' '6' '5'
 '1' 'X']
CAMEO_INTL_2015: [nan 51.0 24.0 12.0 43.0 54.0 22.0 14.0 13.0 15.0 33.0 41.0 34.0 55.0 25.0
 23.0 31.0 52.0 35.0 45.0 44.0 32.0 '22' '24' '41' '12' '54' '51' '44'
 '35' '23' '25' '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45'
 'XX']
CJT_GESAMTTYP: [ 2.  5.  3.  4.  1.  6. nan]
D19_BANKEN_ANZ_12: [0 3 1 2 4 5 6]
D19_BANKEN_ANZ_24: [0 5 1 2 4 3 6]
D19_BANKEN_DATUM: [10  5  8  6  9  1  7  4  2  3]
D19_BANKEN_DIREKT: [0 1 6 5 4 3 7 2]
D19_BANKEN_GROSS: [0 2 6 3 5 1 

KBA13_KW_120: [nan  4.  3.  1.  5.  0.  2.]
KBA13_KW_121: [nan  3.  4.  5.  2.  0.  1.]
KBA13_KW_30: [nan  1.  2.  3.]
KBA13_KW_40: [nan  2.  1.  4.  5.  0.  3.]
KBA13_KW_50: [nan  4.  2.  3.  0.  1.  5.]
KBA13_KW_60: [nan  0.  1.  2.  3.  4.  5.]
KBA13_KW_61_120: [nan  3.  5.  2.  4.  1.]
KBA13_KW_70: [nan  1.  4.  3.  2.  0.  5.]
KBA13_KW_80: [nan  2.  4.  1.  3.  0.  5.]
KBA13_KW_90: [nan  3.  2.  0.  4.  1.  5.]
KBA13_MAZDA: [nan  2.  3.  4.  5.  1.]
KBA13_MERCEDES: [nan  4.  3.  5.  2.  1.]
KBA13_MOTOR: [nan  3.  4.  2.  1.]
KBA13_NISSAN: [nan  2.  3.  5.  4.  1.]
KBA13_OPEL: [nan  3.  2.  4.  1.  5.]
KBA13_PEUGEOT: [nan  4.  3.  2.  1.  5.]
KBA13_RENAULT: [nan  3.  2.  4.  5.  1.]
KBA13_SEG_GELAENDEWAGEN: [nan  2.  5.  3.  4.  1.]
KBA13_SEG_GROSSRAUMVANS: [nan  3.  4.  1.  5.  2.]
KBA13_SEG_KLEINST: [nan  2.  3.  5.  4.  1.]
KBA13_SEG_KLEINWAGEN: [nan  2.  3.  4.  1.  5.]
KBA13_SEG_KOMPAKTKLASSE: [nan  5.  1.  4.  3.  2.]
KBA13_SEG_MINIVANS: [nan  4.  3.  2.  1.  5.]
KBA13_SEG_MI

In [79]:
data_type.loc[data_type['Attribute']==col, 'Type'] #.iloc[0]=='Numerical'

Series([], Name: Type, dtype: object)

In [80]:
col

'D19_BUCH_CD'

In [37]:
cols_to_drop_azdias

['AKT_DAT_KL',
 'ALTERSKATEGORIE_FEIN',
 'ALTER_KIND1',
 'ALTER_KIND2',
 'ALTER_KIND3',
 'ALTER_KIND4',
 'ANZ_KINDER',
 'ANZ_STATISTISCHE_HAUSHALTE',
 'ARBEIT',
 'CJT_KATALOGNUTZER',
 'CJT_TYP_1',
 'CJT_TYP_2',
 'CJT_TYP_3',
 'CJT_TYP_4',
 'CJT_TYP_5',
 'CJT_TYP_6',
 'D19_KONSUMTYP_MAX',
 'D19_LETZTER_KAUF_BRANCHE',
 'D19_SOZIALES',
 'D19_TELKO_ONLINE_QUOTE_12',
 'D19_VERSI_DATUM',
 'D19_VERSI_OFFLINE_DATUM',
 'D19_VERSI_ONLINE_DATUM',
 'D19_VERSI_ONLINE_QUOTE_12',
 'DSL_FLAG',
 'EINGEFUEGT_AM',
 'EINGEZOGENAM_HH_JAHR',
 'EXTSEL992',
 'FIRMENDICHTE',
 'GEMEINDETYP',
 'HH_DELTA_FLAG',
 'KBA13_ANTG1',
 'KBA13_ANTG2',
 'KBA13_ANTG3',
 'KBA13_ANTG4',
 'KBA13_BAUMAX',
 'KBA13_GBZ',
 'KBA13_HHZ',
 'KBA13_KMH_210',
 'KK_KUNDENTYP',
 'KOMBIALTER',
 'KONSUMZELLE',
 'LNR',
 'MOBI_RASTER',
 'RT_KEIN_ANREIZ',
 'RT_SCHNAEPPCHEN',
 'RT_UEBERGROESSE',
 'STRUKTURTYP',
 'UMFELD_ALT',
 'UMFELD_JUNG',
 'UNGLEICHENN_FLAG',
 'VERDICHTUNGSRAUM',
 'VHA',
 'VHN',
 'VK_DHT4A',
 'VK_DISTANZ',
 'VK_ZG11']

In [36]:
customers_coldropped

,AGER_TYP,ALTER_HH,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,...,SHOPPER_TYP,SOHO_KZ,TITEL_KZ,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,2,10.0,1.0,0.0,2.0,0.0,3.0,1A,1,13,...,3,0.0,0.0,1,6.0,9.0,7.0,3,1,4
1,-1,11.0,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,...,3,0.0,0.0,1,0.0,9.0,NaN,3,1,4
2,-1,6.0,1.0,0.0,1.0,0.0,7.0,5D,5,34,...,1,0.0,0.0,2,6.0,9.0,2.0,3,2,4
3,1,8.0,0.0,NaN,0.0,0.0,7.0,4C,4,24,...,0,0.0,0.0,1,NaN,9.0,7.0,1,1,4
4,-1,20.0,7.0,0.0,4.0,0.0,3.0,7B,7,41,...,1,0.0,0.0,2,2.0,9.0,3.0,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191647,1,9.0,1.0,0.0,1.0,0.0,1.0,1C,1,14,...,1,0.0,0.0,2,6.0,9.0,1.0,3,1,3
191648,-1,0.0,0.0,NaN,0.0,0.0,6.0,5B,5,32,...,3,0.0,0.0,2,NaN,9.0,3.0,3,2,4
191649,2,15.0,1.0,0.0,3.0,0.0,7.0,4D,4,24,...,0,0.0,0.0,1,3.0,9.0,3.0,3,1,4
191650,3,13.0,4.0,0.0,4.0,0.0,3.0,4C,4,24,...,2,0.0,0.0,1,4.0,9.0,2.0,3,2,3


## Compare to population

Similar to Kolmogorov-Smirnov Test
1. loop through each column
  1. calculate relative proportions of each category
    - for both population and costumers
  2. Calculate difference for each
  3. Sum the absolute value
  
2. Order columns by the biggest difference 

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')